In [ ]:
from multiprocessing import Manager
from threading import Thread
import pickle
import json
from googleapiclient import discovery

manager = Manager()

In [ ]:
# Replace with your Google API key
GOOGLE_API_KEY = '<enter api key here>'

# Define the URL for the Google Cloud Natural Language API
url = "https://language.googleapis.com/v2/documents:moderateText"

client = discovery.build(
  "commentanalyzer",
  "v1alpha1",
  developerKey=GOOGLE_API_KEY,
  discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
  static_discovery=False,
)

analyze_request = {
  'comment': { 'text': 'friendly greetings from python' },
  'requestedAttributes': {'TOXICITY': {}}
}

In [ ]:
# load data
with open('./data/comments.pkl', 'rb') as handle:
    comments = pickle.load(handle)

comments = list(set(comments))

In [ ]:
moderation_map = manager.dict()

def moderation(local_dictionary, texts):
    for comment in texts:
      response = ''
      while response == '':
          # try catch to ensure outputs are obtained
          try:
              response = client.comments().analyze(body=analyze_request).execute()
              local_dictionary[comment] = json.dumps(response, indent=2)
          except Exception as e:
            print(e)
            response = ''

In [ ]:
num_threads = 4
threads = []
for i in range(0, num_threads):
    start = i * (len(comments) // num_threads)
    end = (i + 1) * (len(comments) // num_threads) if i != num_threads - 1 else len(comments)
    thread = Thread(target=moderation,args=(moderation_map, comments[start:end], ))
    threads += [thread]
    thread.start()

for thread in threads:
    thread.join()

In [ ]:
moderation_map = dict(moderation_map)
with open('./results/comments_moderation_google_perspective.pkl', 'wb') as handle:
    pickle.dump(moderation_map, handle, protocol=pickle.HIGHEST_PROTOCOL)